In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#pandas module used to load the train.csv table
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

#.head() returns the first five rows of the table
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#same procedure as we did with the test file
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
#check if assuming all female passengers survived is a reasonable guess
#looking at the survived column of the data
women = train_data.loc[train_data.Sex == 'female']["Survived"]
# the number of women survived divided by the total number of women
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


therefore, if your model assumed that all female passengers survived, it would guess right for about 75% of female passengers.

In [5]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)


% of men who survived: 0.18890814558058924


If your model assumed that all male passengers did not sruvive, it would guess right for about 81% of male passengers. This is not a bad first guess, but we can do better by considering more columns.

I'll try using a random forest model. Each tree will individually consider each passenger's data and vote on whether the individual survivied. Then, the model decides that the outcome with the most votes wins. Below is an example of how a random forest works:![ex of how a random forest works](https://i.imgur.com/AC9Bq63.png)

In [6]:
from sklearn.ensemble import RandomForestClassifier
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]

#get_dummies creates dummy variables from your categorical/factor variables. utilizes one hot encoding
"""
ex: if the column "sex" was full of "male" and "female" values, 
it would get rid of that column and create new columns "sex_male" and "sex_female".
if an observation had "male" in the "sex" column, it now has a 1 in the "sex_male" column
and a 0 in "sex_female" column.
"""
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

"""
For RandomForestClassifier:
n_estimators: number of trees in forest
max_depth: max depth of tree
random_state: set the seed so model always produce same results
"""
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
